<font color = magenta> Lixandru Andreaa-Bianca Grupa10LF382
<br> 
<font color = magenta> Pepene Adina-FLorentina Grupa10LF382

# Laborator 9

## Modele de regresie

Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru cautarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



*Notare:* rezolvarea va fi incarcata pe platforma de elearning in saptamana 11-15 mai.

# Citirea si separarea datelor 

In [50]:
import pandas as pd 
import numpy as np
from typing import List

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from statistics import mean 

In [51]:
dataframe_cpu_names = ['vendor_names', 'model', 'myct', 'mmin', 'mmax', 'cach', 'chmin', 'chmax', 'prp', 'erp']

dataframe_cpu = pd.read_csv('data\machine\machine.data', names=dataframe_cpu_names, header=None)

dataframe_cpu = dataframe_cpu.drop(columns=['vendor_names', 'model', 'erp'])

dataframe_cpu.head()

,myct,mmin,mmax,cach,chmin,chmax,prp
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [52]:
dataframe_cpu_x = dataframe_cpu.values[:, :-1]
dataframe_cpu_y = dataframe_cpu.values[:, -1]

In [53]:
dataframe_housing_names = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis',
                           'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']

dataframe_housing = pd.read_csv('data\housing\housing.data', names=dataframe_housing_names, header = None, sep=r"\s+")

dataframe_housing.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [54]:
dataframe_housing_x = dataframe_housing.values[:, :-1]
dataframe_housing_y = dataframe_housing.values[:, -1]

# Cautarea hiperparametrilor 

In [55]:
def search_grid(model, parameters:dict , x:np.ndarray, y:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
    
    It returns the errors.
    '''
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)
    
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='neg_mean_squared_error', cv=3,  return_train_score=True)
    grid_search.fit(x_train, y_train)
    
    return errors(grid_search, x, y)

In [56]:
def search_random(model, parameters:dict , x:np.ndarray, y:np.ndarray):
    
    '''This function takes the parameters and search for best parameters.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
    
    It returns the errors.
    '''
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)
    
    rand_search = RandomizedSearchCV(estimator=model, param_distributions=parameters, scoring='neg_mean_squared_error', cv=3,  return_train_score=True)
    rand_search.fit(x_train, y_train)
    
    return errors(rand_search, x, y)

# Calcularea erorilor 

In [57]:
def errors(model, x:np.ndarray, y:np.ndarray)-> List[float]:
    
    '''Errors function calculates mean absolute error, 
    mean squared error and median absolute error for model.
    
    Parameters:
        model - model from sklearn 
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test
        
    Returns a list with mean of errors:
    neg_mean_absolute_error for train and test data
    neg_mean_squared_error for train and test data
    neg_median_absolute_error for train and test data
    '''
    
    scores = cross_validate(model, x, y, cv = 5, scoring = ['neg_mean_absolute_error',
                                                            'neg_mean_squared_error',
                                                            'neg_median_absolute_error'], return_train_score = True)
    
    result = [(-1)*scores['train_neg_mean_absolute_error'].mean(), (-1)*scores['train_neg_mean_squared_error'].mean(),
              (-1)*scores['train_neg_median_absolute_error'].mean(), (-1)*scores['test_neg_mean_absolute_error'].mean(),
              (-1)*scores['test_neg_mean_squared_error'].mean(), (-1)*scores['test_neg_median_absolute_error'].mean(),
              scores['fit_time'].mean(), scores['score_time'].mean()]
    
    return result

# K-Nearest Neighbor Regression(KNN)

In [58]:
def KNN(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''KNN function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test 
    '''
    
    print ("KNN", search_grid(model, parameters, x, y))
    print ()
    print ("KNN", search_random(model, parameters, x, y))    

# Decision Tree Regression

In [59]:
def decision_tree(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''decision_tree function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test 
    '''
    
    print ("Decision tree", search_grid(model, parameters, x, y))
    print ()
    print ("Decision tree", search_random(model, parameters, x, y))    

# Random Forest Regression

In [60]:
def random_forest(model, parameters:dict, x:np.ndarray, y:np.ndarray):
    
    '''random_forest function get params and do regression based on k-nearest
    neighbors with best parameters generated by grid search and random search.
    It generates the error values.
    
    Parameters:
        model - model from sklearn 
        parameters - all the alternatives to the hyperparameters
        x_train, y_train - input and output values for model train
        x_test, y_test - input and output values for model test 
    '''
    
    print ("Random Forest", search_grid(model, parameters, x, y))
    print ()
    print ("Random Forest", search_random(model, parameters, x, y))    

# Aplicarea modelelor pentru generarea erorilor 

In [61]:
def start_knn(x:np.ndarray, y:np.ndarray):
    
    model_knn = KNeighborsRegressor()
    parameters_knn = {'n_neighbors': [3, 4, 5, 6, 8, 9, 10, 11], 'p': [2, 3, 4, 5]}
    KNN(model_knn, parameters_knn, x, y)

In [62]:
def start_decision_tree(x:np.ndarray, y:np.ndarray):
    
    model_decision_tree = DecisionTreeRegressor()
    parameters_decision_tree = {'criterion': ['mse', 'friedman_mse'], 'max_depth': [3, 4, 5, 6, 7, 8]}
    decision_tree(model_decision_tree, parameters_decision_tree, x, y)

In [63]:
def start_random_forest(x:np.ndarray, y:np.ndarray):
    
    model_random_forest = RandomForestRegressor()
    parameters_random_forest = {'criterion': ['mse', 'friedman_mse'], 'n_estimators': [ 80, 100, 120], 'max_depth': [3, 4, 5, 6]}
    random_forest(model_random_forest, parameters_random_forest, x, y)

In [64]:
def start(x:np.ndarray, y:np.ndarray):
    
    start_knn(x, y)
    start_decision_tree(x, y)
    start_random_forest(x, y)

In [65]:
start(dataframe_cpu_x, dataframe_cpu_y)

KNN [22.433014328485886, 1888.191426521956, 10.633333333333335, 37.1522841656988, 5296.563042005419, 16.7, 0.41849322319030763, 0.0012000083923339843]

KNN [25.108545409181637, 2736.2748727247886, 11.277777777777779, 37.41859594786424, 6201.684593173312, 19.22222222222222, 0.1146934986114502, 0.001204061508178711]
Decision tree [11.537344192897303, 367.0380831348116, 7.162266955266955, 46.37520434477768, 11769.19714853405, 19.33429014844804, 0.05804500579833984, 0.000597381591796875]

Decision tree [12.505044921588851, 397.57085968261964, 7.292319902319901, 46.17576512776863, 10800.196206094008, 19.04793477764066, 0.049481773376464845, 0.0011894702911376953]
Random Forest [16.016839340420148, 723.6304931736577, 9.517280417391538, 35.53333120727753, 7342.680620135303, 17.66126969282366, 8.337134885787965, 0.004780101776123047]

Random Forest [15.596234304797647, 671.0788877871073, 9.462529819249307, 34.82290735377772, 7270.119425742203, 16.938758971633852, 3.503833293914795, 0.005984640

In [66]:
start(dataframe_housing_x, dataframe_housing_y)

KNN [3.477460383075344, 25.894476371210637, 2.36909090909091, 6.312175994069995, 79.68782878553094, 4.786969696969697, 1.240891695022583, 0.002971458435058594]

KNN [3.4387129083278696, 25.51810021004794, 2.3360606060606064, 6.371005877058295, 82.29270462766064, 4.686363636363636, 0.3853675365447998, 0.00458059310913086]
Decision tree [1.837098123512017, 6.237308781118562, 1.4197181616832786, 3.8132935430312216, 37.12047422148272, 2.6283925925925935, 0.06442809104919434, 0.0006011962890625]

Decision tree [1.7260796030329846, 5.717833303169307, 1.311309178743964, 3.771791236480026, 37.902366529133964, 2.448041159790571, 0.05726361274719238, 0.0006005287170410157]
Random Forest [1.5279580135471353, 4.004009657761555, 1.228048786811763, 3.0894302869998223, 22.384089831064966, 2.0788219672470256, 10.919804048538207, 0.008178806304931641]

Random Forest [1.4263466616235128, 3.463485301763587, 1.141258133088307, 3.0339772103026803, 21.22063165767173, 2.1382960947978726, 4.543882083892822, 0

# Documentatie

## 1. K-Nearest Neighbors Regression(KNN)

K-Nearest Neighbors Regression este un algoritm simplu care stochează toate cazurile posibile și prezice o valoare pe baza unei măsuri de similaritate (de exemplu, funcții de distanță). KNN a fost utilizat în estimarea statistică și recunoașterea modelului de la începutul anilor 1970 ca tehnică non-parametrică. Regresia KNN utilizează aceleași funcții de distanță ca și clasificarea KNN. K reprezinta numarul de vecini care este cautat pentru a-l determina pe cel mai potrivit.
<br><br>Când __K = 1__ algoritmul este cunoscut ca nearest neighbor. Acesta este cel mai simplu caz. Sa presupunem că P1 este un punct pentru care trebuie sa ii prezicem eticheta. Mai întâi, găsim cel mai apropiat punct de P1 și atribuim eticheta acestuia lui P1.

<img src="resources/image1.png" width="400">

Un alt exemplu pentru __K = 3__ este:

<img src="resources/image2.png" width="400">

Urmarind plot-urile de mai jos putem observa ca folosind doar un singur vecin, fiecare punct din setul de antrenare are o influență evidentă asupra predicțiilor, iar valorile prezise trec prin toate punctele. Aceasta duce la o predicție foarte nesigură. Considerând mai mulți vecini se ajunge la predicții mai ușoare, dar acestea nu se potrivesc și cu datele de instruire.

<img src="resources/image3.png" width="1000">

Regresia K-Nearest Neighbors poate fi utilizată în cazurile în care etichetele de date sunt continue, mai degrabă decât variabile discrete. Eticheta atribuită unui punct de interogare este calculată pe baza mediilor etichetelor celor mai apropiați vecini.
În faza de clasificare, k este o constantă definită de utilizator, iar un vector nemarcat (o interogare sau un punct de testare) este clasificat prin atribuirea etichetei care este cea mai frecventă dintre eșantioanele de pregătire k cele mai apropiate de punctul de interogare.

__Metode de calcul al distantei dintre doua puncte__

Algoritmul KNN este un algoritm de invatare supervizata, care foloseste distante pentru a gasi similitudini si diferente.

__1. Distanta Euclidiana:__ este cea mai frecventă și măsoară distanța liniară dintre două probe
__2. Distanta Manhattan:__ măsoară timpul de călătorie punct-la-punct și este utilizata în mod obișnuit pentru predictorii binari

<img src="resources/image4.png" width="250">

__3. Distanta Hamming__

<img src="resources/image5.png" width="150">


Distanta Euclidiana si distanta Manhattan sunt folosite pentru variabile continue, iar distanta Hamming este folosita pentru variabile categoriale.

Cum aflam care este cel mai potrivit K?
Daca folosim un K foarte mic vom face overfitting pe setul de antrenare si nu vom avea rezultate deloc bune pentru setul de testare. Daca folosim un K foarte mare nu vom obtine rezultate bune nici pe setul de antrenare, nici pe setul de intrare. Ce facem? Solutia este curba 'elbow curve'. Un K bun poate fi selectat prin diferite tehnici euristice (ex: optimizarea hiperparametrului). 

<img src="resources/image6.png" width="550">

Un dezavantaj al clasificării de bază a „votului majoritar” apare atunci când exemple dintr-o clasă mai frecventă tind să domine predicția noului exemplu, deoarece acestea tind să fie comune printre cei mai apropiați k din cauza numărului mare. O modalitate de a depăși această problemă este ponderea clasificării, ținând cont de distanța de la punctul de testare la fiecare dintre cei mai apropiați k vecini ai săi. Clasa (sau valoarea, în problemele de regresie) din fiecare dintre cele mai apropiate k puncte este înmulțită cu o greutate proporțională cu inversul distanței de la acel punct la punctul de testare. 

Precizia algoritmului KNN poate fi sever degradată prin prezența unor caracteristici zgomotoase sau irelevante sau dacă scările de caracteristici nu sunt în concordanță cu importanța lor. 

### Bibliografie:

1. https://www.saedsayad.com/k_nearest_neighbors_reg.htm
2. https://medium.com/analytics-vidhya/k-neighbors-regression-analysis-in-python-61532d56d8e4
3. https://scikit-learn.org/stable/modules/neighbors.html#regression
4. https://www.analyticsvidhya.com/blog/2018/08/k-nearest-neighbor-introduction-regression-python/
5. https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm

## 2. Decision Tree Regression

Arborele decizional construiește modele de regresie sau clasificare sub forma unei structuri de arbori. Acesta descompune un set de date în subseturi mai mici și mai mici, în același timp se dezvoltă treptat un arbore de decizie asociat. Rezultatul final este un arbore cu noduri de decizie și noduri frunze. Un nod de decizie are două sau mai multe ramuri (de exemplu, însorit, înnorat și ploios), fiecare reprezentând valori pentru atributul testat. Nodul frunza (de exemplu, ore jucate) reprezintă o decizie cu privire la ținta numerică. Cel mai înalt nod decizie într-un arbore corespunde celui mai bun predictor numit nod rădăcină. Arborii de decizie pot gestiona atât date categorice, cât și numerice. Nodurile de decizie reprezinta conditiile, iar nodurile frunza reprezinta rezultatele.

<br>
Exemplul de mai jos care arată un arbore de decizie care determina cel mai mic numar dintre trei elemente.

<img src="resources/image8.jpg" width="650">

Regresia arborelui decizional observă trasaturile unui obiect și ataseaza în structura unui arbore trasaturile pentru a prezice date în viitor. Ieșirea continuă înseamnă că ieșirea / rezultatul nu este discret, adică nu este reprezentat doar de un set discret, cunoscut de numere sau valori.

_Exemplu de ieșire discreta:_ un model de predicție a vremii care prevede dacă va fi sau nu ploaie într-o anumită zi.
<br>_Exemplu de ieșire continuă:_ model de predicție a profitului care afirmă profitul probabil care poate fi generat din vânzarea unui produs.

Pornind de la rădăcină, datele sunt împărțite pe caracteristicile care au ca rezultat cel mai mare câștig de informații (IG- Information Gain). Într-un proces iterativ, repetăm această procedură de divizare la fiecare nod copil până când frunzele sunt pure - adică eșantioanele la fiecare nod aparțin aceleiași clase.

În practică, acest lucru poate duce la un arbore foarte adânc, cu numeroase noduri, ceea ce poate duce cu ușurință la supraîncadrare. Astfel, de obicei dorim să tăiem copacul prin stabilirea unei limite pentru adâncimea maximă a copacului.

Pentru a împărți nodurile in cele mai relevante caracteristici, trebuie să definim o funcție obiectivă pe care dorim să o optimizăm prin algoritmul de învățare. Aici, funcția noastră obiectivă este de a maximiza câștigul de informații la fiecare divizare, pe care o definim astfel:

<img src="resources/image9.png" width="500">

Pentru a utiliza un arbore de decizie pentru regresie, avem nevoie de o măsurătoare care să fie potrivită pentru variabile continue, astfel încât să definim măsura de impuritate folosind eroarea medie pătrată (MSE) a nodurilor copiilor:

<img src="resources/image10.png" width="250">

Aici, Nt este numărul de eșantioane de antrenament al nodului t, Dt este subsetul de formare al nodul t, y (i) este adevărata valoare și ŷt este valoarea țintă prevăzută:

<img src="resources/image11.png" width="125">

În practică, este important să stii cum să alegi o valoare adecvată pentru o adâncime a unui arbore pentru a face overfit sau a face undefit. Să știi cum să combini arbori de decizie pentru a forma o pădure aleatoare este, de asemenea, util, intrucat de obicei, are o performanță de generalizare mai bună decât un arbore de decizie individual, ceea ce ajută la scăderea variației modelului. De asemenea, este mai puțin sensibil la valorile din setul de date și nu necesită prea multe reglări de parametri.

### Bibliografie

1. https://www.geeksforgeeks.org/python-decision-tree-regression-using-sklearn/
2. https://towardsdatascience.com/https-medium-com-lorrli-classification-and-regression-analysis-with-decision-trees-c43cdbc58054 -> foarte relevant
3. https://saedsayad.com/decision_tree_reg.htm

## 3. Random Forest Regression

Random Forest este o tehnică de ansamblu capabilă să îndeplinească atât sarcini de regresie, cât și de clasificare cu utilizarea mai multor arbori de decizie și o tehnică numită Bootstrap Aggregation, cunoscută de obicei ca bagging. Bagging, în metoda Random Forest, implică instruirea fiecărui arbore de decizie pe un eșantion de date diferit, unde sampling-ul se face cu înlocuire.

Arborii de decizie sunt o metodă populară pentru diferite metode de machine learning.
În special, arborii care sunt foarte adânci tind să învețe tipare extrem de neregulate: își potrivesc seturile de antrenament ai auvariație foarte mare. Random Forest este o modalitate de medie a mai multor arbori de decizie profundă, instruiți pe diferite părți ale aceluiași set de antrenament, cu scopul de a reduce variația. 

Procedura de bootstrapping duce la o performanță mai bună a modelului, deoarece scade variația modelului, fără a crește bias-urile. Aceasta înseamnă că, deși iesirile unui singur arbore sunt foarte sensibile la zgomot în setul său de formare, media mai multor arbori nu este, atâta timp cât nu exista corelații. Pur și simplu antrenarea multor arbori pe un singur set de antrenament ar oferi arbori puternic corelați (sau chiar același arbore de multe ori, dacă algoritmul de antrenare este determinist); eșantionarea bootstrap este o modalitate de a corela copacii arătându-le diferite seturi de antrenament.

<img src="resources/image12.jpg" width="400">

Metoda de mai sus descrie algoritmul de bagaj original pentru arbori. Random Forest diferă într-un singur mod de această schemă generală: folosesc un algoritm de învățare arbore modificat care selectează, la fiecare împărțire a candidatului în procesul de învățare, un subset aleator al caracteristicilor. Acest proces este uneori numit „bagaj de funcții”(feature bagging). Motivul pentru a face acest lucru este corelația arborilor dintr-un eșantion de bootstrap obișnuit: dacă una sau câteva caracteristici sunt predictori foarte puternici pentru variabila de răspuns(ieșire țintă), aceste caracteristici vor fi selectate în multe dintre arborele B, cauzându-le a deveni corelat. O analiză a modului în care bagajul și proiecția aleatorie a subspațiului contribuie la câștigurile de precizie în diferite condiții.

Diferite tipuri de modele au avantaje diferite. Modelul Random Forest este foarte bun în tratarea datelor tabulare cu caracteristici numerice sau caracteristici categorice cu mai puțin de sute de categorii. Spre deosebire de modelele liniare, Random Forest este capabil să capteze interacțiuni neliniare între caracteristici și țintă.

O notă importantă este că modelele bazate pe arbori nu sunt proiectate să funcționeze cu caracteristici foarte rare. Atunci când avem de a face cu date de intrare reduse(de exemplu, caracteristici categorice cu dimensiuni mari), putem prelucra în prealabil funcțiile rare pentru a genera statistici numerice, sau putem trece la un model liniar, care este mai potrivit pentru astfel de scenarii.

### Bibliografie

1. https://medium.com/datadriveninvestor/random-forest-regression-9871bc9a25eb
2. https://en.wikipedia.org/wiki/Random_forest
3. https://turi.com/learn/userguide/supervised-learning/random_forest_regression.html